In [61]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [62]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [63]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(50000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [64]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
R1656N93KD0BTV,B005CUKHLK,Hershey's Special Dark Chocolate Bar,5,0,0,N,Y,Five Stars,Five Stars. Mom loves them,0.000000,0.000000,NaN
R29LCDZ9UOWS6H,B00UCETD5M,Eight O'Clock Coffee Colombian Peaks Keurig 2....,5,0,0,N,Y,Five Stars,Five Stars. Good strong great-quality coffee!,0.620833,0.666667,NaN
R27SDQR9M1OLNM,B001SAWE5U,Land-O-Lakes Mint Hot Cocoa Mix 15 oz (Pack of...,5,0,0,N,Y,Five Stars,Five Stars. Bomb,0.000000,0.000000,NaN
R1HY7V1Y5JQ8YD,B0051WV5K6,Keysands Live Organic Water Kefir Grains (Crys...,5,1,1,N,Y,Very fast shipping,Very fast shipping. Grains were shipped fast a...,0.440000,0.590000,NaN
R2HPQMF11D5LVQ,B0027AREDC,Hostess Ho-Hos 2 boxes 20 cakes,5,0,1,N,Y,A classic,A classic. Always awesome. Already in the free...,0.600000,0.656250,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
R2MZBS7958YDZW,B001EPPBB8,De Cecco Pasta,5,1,1,N,Y,Love It,"Love It. This is the best afford, everyday pas...",0.441667,0.533333,NaN
R392OZOJJMX7SH,B000MN630Q,Taylors of Harrogate Assorted Tea,5,0,0,N,Y,great tea,great tea. one of the best &#34;earl gray&#34;...,1.000000,0.300000,NaN
R24ZBMOEE70PQH,B005YUB0C8,"Crown Prince Sardines, 4.25-Ounce Cans (Pack o...",1,2,2,N,Y,Horrid after-taste and difficult to open can,Horrid after-taste and difficult to open can. ...,-0.025714,0.166667,NaN


In [65]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [66]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

neg_reviews["tokened_review"] = np.nan
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    neg_reviews.loc[index, "tokened_review"] = str(new_text)
    

<ipython-input-66-4c7ba725551c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = np.nan
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [67]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale"]
for word in hazardous_words:
    neg_reviews.loc[[word in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

In [68]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

473

In [69]:
non_hazardous = neg_reviews.query("health_hazard != 1 and polarity < 0", engine="python").sample(num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [70]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.12445363,
        0.        ],
       [0.        , 0.1341244 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.17582202,
        0.        ]])

In [91]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [92]:
#from sklearn.naive_bayes import GaussianNB
#classifier = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9071729957805907


In [93]:
trial = neg_reviews.sample(100)

In [94]:
matrix.get_feature_names()

['actual',
 'ad',
 'add',
 'almost',
 'also',
 'amazon',
 'ani',
 'anoth',
 'anyth',
 'arriv',
 'artifici',
 'aw',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'batch',
 'bean',
 'becaus',
 'befor',
 'better',
 'bit',
 'bitter',
 'black',
 'bottl',
 'bought',
 'box',
 'brand',
 'broken',
 'buy',
 'came',
 'candi',
 'cant',
 'case',
 'chocol',
 'coffe',
 'come',
 'compani',
 'contain',
 'cooki',
 'could',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'end',
 'even',
 'ever',
 'everi',
 'expect',
 'expens',
 'expir',
 'extrem',
 'fairi',
 'find',
 'first',
 'flavor',
 'food',
 'found',
 'fresh',
 'gave',
 'get',
 'gift',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'high',
 'hope',
 'horribl',
 'howev',
 'ill',
 'im',
 'ingredi',
 'item',
 'ive',
 'keep',
 'kind',
 'know',
 'last',
 'least',
 'like',
 'littl',
 'local',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'mani',
 'm

In [102]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier.predict(trial_X)
trial_y

array([0., 0., 1., ..., 0., 1., 0.])

In [107]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-107-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [108]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
RV47XO75DH2XJ,B00BG0ZJ0Y,Matcha DNA Certified Organic Matcha Green Tea,1,2,12,N,N,A VERY BAD BUY!,A VERY BAD BUY!. This is not &#34;100% USDA or...,-0.020000,0.200000,0.0,"['veri', 'bad', 'buy', 'thi', '34100', 'usda',..."
RA9YT0A7MWWQZ,B004QQ806A,Cook's Bourbon Extract,3,4,4,N,Y,"Nice flavor, but weak","Nice flavor, but weak. There aren't many place...",-0.024444,0.665556,0.0,"['nice', 'flavor', 'weak', 'arent', 'mani', 'p..."
R2BFW4SUA2FGFM,B002AQP6AQ,Betty Crocker Asian Chicken Helper Lo Mein,1,1,11,N,N,Really,"Really. I tried This Stuff Not Once But Twice,...",-0.237500,0.733333,1.0,"['realli', 'tri', 'thi', 'stuff', 'onc', 'twic..."
R38QZCSVGSV0EY,B000DZE73S,Da Vinci Gourmet Syrups Syrup Pump,1,4,6,N,Y,Doesn't fit the Subscribe & Save 750ml Da Vinc...,Doesn't fit the Subscribe & Save 750ml Da Vinc...,0.167593,0.206070,0.0,"['doesnt', 'fit', 'subscrib', 'save', '750ml',..."
R1M107HGYOIW1F,B005V9YXTO,"Eat Your Vegetables Sea Salt Veggie Chips, 4.5...",3,0,0,N,N,they are good altho you can find them at many ...,they are good altho you can find them at many ...,0.155556,0.325000,0.0,"['good', 'altho', 'find', 'mani', 'grocer', 'e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R37TXQ4ZD4Z9AB,B000F4J73W,Twinings Prince of Wales Tea,1,0,0,N,Y,TERRIBLE for a great TEA COMPANY,TERRIBLE for a great TEA COMPANY. This tea was...,-0.333333,0.333333,1.0,"['terribl', 'great', 'tea', 'compani', 'thi', ..."
R351ZDA49ZAL5M,B00I7V5Q0I,Organic Cassia Cinnamon Ground,3,0,1,N,Y,Three Stars,Three Stars. Disappointed. Taste the same as r...,-0.375000,0.425481,0.0,"['three', 'star', 'disappoint', 'tast', 'regul..."
R1L97B7MVU2EJB,B00WSZ4EFC,"Beanitos Black Bean Chips with Sea Salt, 6 Oun...",3,0,0,N,N,"But they were nice,","But they were nice,. I received these as part ...",0.365104,0.697917,0.0,"['nice', 'receiv', 'part', 'influenc', 'vox', ..."


In [109]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R2BFW4SUA2FGFM,B002AQP6AQ,Betty Crocker Asian Chicken Helper Lo Mein,1,1,11,N,N,Really,"Really. I tried This Stuff Not Once But Twice,...",-0.237500,0.733333,1.0,"['realli', 'tri', 'thi', 'stuff', 'onc', 'twic..."
R262LY37YJH3V3,B0006JHR5Q,Melon Pure Cane Syrup,3,0,0,N,Y,Three Stars,Three Stars. Hoping it was like the one Star B...,0.000000,0.000000,1.0,"['three', 'star', 'hope', 'wa', 'like', 'one',..."
R19M674XY56FNL,B00J0HTLJU,"Bobs Red Mill White Rice Flour, 25 Pound -- 1 ...",3,0,1,N,Y,I was disappointed when shipment arrived,I was disappointed when shipment arrived. I wa...,-0.185000,0.470000,1.0,"['wa', 'disappoint', 'shipment', 'arriv', 'wa'..."
R3CRIZXK27MSK7,B0025UCGCI,"Armour Chili No Beans, 8-Ounce Pouches (Pack o...",2,0,0,N,Y,Super salty,Super salty. We eat a lot of chili in the wint...,0.207440,0.407440,1.0,"['super', 'salti', 'eat', 'lot', 'chili', 'win..."
R3IWWOHQLJAMSX,B002UKT9UU,Classic Buttery Popover Mix (2 lbs),1,0,0,N,Y,One Star,One Star. Family didn't care for this mix very...,0.200000,0.300000,1.0,"['one', 'star', 'famili', 'didnt', 'care', 'th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
RQZWNRVHKM8GX,B001E0JMWC,Torani Sauces,1,2,3,N,Y,Sauces,Sauces. Looking for low carb...this is not but...,0.166667,0.408333,1.0,"['sauc', 'look', 'low', 'carbthi', 'came', 'se..."
REIARMTI9TQ8I,B001EQ54RS,"Diamond Walnuts, Finely Diced, 10-Ounce Bags (...",2,1,1,N,Y,""" Not happy with the wrong type of walnut but ...",""" Not happy with the wrong type of walnut but ...",-0.069922,0.309028,1.0,"['happi', 'wrong', 'type', 'walnut', 'order', ..."
R30FLU3OL9MV0X,B0028SUEOO,"Seafoodxpress-Fresh Thresher Shark Fillets, 2lbs",2,7,10,N,N,"I love seafood, but at the same time everyone ...","I love seafood, but at the same time everyone ...",0.086111,0.486944,1.0,"['love', 'seafood', 'time', 'everyon', 'need',..."
